In [210]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [211]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
#    def parse(cls, s, testnet=False):
#        version = little_endian_to_int(s.read(4))
#        num_inputs = read_varint(s)
#        inputs = []
#        for _ in range(num_inputs):
#            inputs.append(TxIn.parse(s))
#        num_outputs = read_varint(s)
#        outputs = []
#        for _ in range(num_outputs):
#            outputs.append(TxOut.parse(s))
#        locktime = little_endian_to_int(s.read(4))
#        return cls(version, inputs, outputs, locktime, testnet=testnet)
# print(transaction)
# tx: 452c629d67e41baec3ac6f04fe744b4b9617f8f859c63b3002f8684e7a4fee03
# version: 1
# tx_ins:
# d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81:0
# tx_outs:
# 32454049:OP_DUP OP_HASH160 bc3b654dca7e56b04dca18f2566cdaf02e8d9ada OP_EQUALVERIFY OP_CHECKSIG
# 10011545:OP_DUP OP_HASH160 1c4bc762dd5423e332166702cb75f40df79fea12 OP_EQUALVERIFY OP_CHECKSIG
# locktime: 410393

#     def fee(self):
#         '''Returns the fee of this transaction in satoshi'''
# input_sum, output_sum = 0, 0
# for tx_in in transaction.tx_ins:
#     input_sum += tx_in.value('https://blockstream.info/api')
# print(input_sum)
# 42505594

# for tx_out in transaction.tx_outs:
#     output_sum += tx_out.amount
# print(output_sum)
# 42465594
# return input_sum - output_sum
print(transaction.fee())
print(transaction.fee() >= 0)

40000
True


In [212]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [213]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [214]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [215]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [216]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.206s

OK


In [217]:
# decode_base58: We need a way to take an address and get the 20-byte hash out of it
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
# print(prev_tx)
# b'\ro\xe5!<\x0b2\x91\xf2\x08\xcb\xa8\xbf\xb5\x9btv\xdf\xfa\xccN\\\xb6on\xb2\n\x08\x08C\xa2\x99'

prev_index = 13

tx_in = TxIn(prev_tx, prev_index)
# print(tx_in)
# 0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13

tx_outs = []

# The amount must be in satoshis; given there are 100,000,000 satoshis per BTC, we have to multiply and cast to an integer.
change_amount = int(0.33*100000000)
# print(change_amount)
# 33000000

# We get what number is encoded in this Base58 address.
# Once we have the number, we convert it to big-endian bytes.
# The first byte is the network prefix and the last 4 are the checksum. The middle 20 are the actual 20-byte hash (aka hash160).
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
# print(change_h160)
# b'\xd5*\xd7\xca\x9b=\tj8\xe7R\xc2\x01\x8eo\xbc@\xcd\xf2o'

change_script = p2pkh_script(change_h160)
# print(change_script)
# OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG

change_output = TxOut(amount=change_amount, script_pubkey=change_script)
# print(change_output)
# 33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG

target_amount = int(0.1*100000000)
# print(target_amount)
# 10000000

target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
# print(target_h160)
# b"P{'A\x1c\xcf\x7f\x16\xf1\x02\x97\xdel\xef?)\x16#\xed\xdf"

target_script = p2pkh_script(target_h160)
# print(target_script)
# OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG

target_output = TxOut(amount=target_amount, script_pubkey=target_script)
# print(target_output)
# 10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG

# We have to designate which network to look up using the testnet=True argument.
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)

# We have created the actual transaction, but every ScriptSig in this transaction is currently empty. Filling it is where we turn next.
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [218]:
from ecc import PrivateKey
from helper import SIGHASH_ALL

# If we have the private key whose public key hash160s to the 20-byte hash in the ScriptPubKey, we can sign z and produce the DER signature
# tx.py def sig_hash(self, input_index):
z = transaction.sig_hash(0)
# print(transaction)
# tx: 3fad891a9d0deda19862a073d35611d57089377adb4585e6c457bcb0f4470821
# version: 1
# tx_ins:
# d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81:0
# tx_outs:
# 32454049:OP_DUP OP_HASH160 bc3b654dca7e56b04dca18f2566cdaf02e8d9ada OP_EQUALVERIFY OP_CHECKSIG
# 10011545:OP_DUP OP_HASH160 1c4bc762dd5423e332166702cb75f40df79fea12 OP_EQUALVERIFY OP_CHECKSIG
# locktime: 410393
#print(z)
# 18037338614366229343027734445863508930887653120159589908930024158807354868134

private_key = PrivateKey(secret=8675309)
# print(private_key)
# print(encode_base58_checksum(private_key))
# def encode_base58(s) return prefix + result
# def encode_base58_checksum(s) return encode_base58(s + hash256(s)[:4])
    
# print(private_key.sign(z))
# Signature(7db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11,10178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a)
der = private_key.sign(z).der()
# print(der)
# b'0D\x02 }\xb2@*3\x11\xa3\xb8E\xb08\x88^=\xd8\x89\xc0\x81&\xa8W\x0f&\xa8D\xe3\xe4\x04\x9cH*\x11\x02 \x10\x17\x8c\xdc\xa4\x12\x9e\xac\xbe\xab|Dd\x8b\xf5\xac\x1f\x9c\xac!|\xd6\t\xd2\x16\xec.\xbc\x8d$,\n'

# SIGHASH_ALL = 1
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
# print(sig)
# b'0D\x02 }\xb2@*3\x11\xa3\xb8E\xb08\x88^=\xd8\x89\xc0\x81&\xa8W\x0f&\xa8D\xe3\xe4\x04\x9cH*\x11\x02 \x10\x17\x8c\xdc\xa4\x12\x9e\xac\xbe\xab|Dd\x8b\xf5\xac\x1f\x9c\xac!|\xd6\t\xd2\x16\xec.\xbc\x8d$,\n\x01'

sec = private_key.point.sec()
# print(sec)
# b'\x03\x93U\x81\xe5,5L\xd2\xf4\x84\xfe\x8e\xd8:\xf7\xa3\tp\x05\xb2\xf9\xc6\x0b\xffq\xd3[\xd7\x95\xf5Kg'

script_sig = Script([sig, sec])
# print(script_sig)
# 304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a01 03935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67

transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [219]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int

secret = little_endian_to_int(hash256(b'i wish you good luck in your tribulations'))
# print(secret)
# 15823921174355222012952670740863432824567694393507537176457013471334478680091

private_key = PrivateKey(secret)
# print(private_key.point.address)
# <bound method S256Point.address of S256Point(a5c4de8e42c0de6697a0c8731f862e21e1c0958da24bb7b44bad618cb31fcb40, b215ccc961657f027d554e9c4f066ca177463c3092e0f749f4b9a797ecbaf88d)>
print(private_key.point.address(testnet=True))

mj3y1sNvb1iw3QS6FsMfTuW4aA47fPntnR


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [220]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.170s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [300]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

prev_tx = bytes.fromhex('c2d650f06d68837df120a052e06394a7b9792b9c935f7376aac49df3e735a6de')
prev_index = 0
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_amount = 0.60 * 0.01557121
change_address = 'mj3y1sNvb1iw3QS6FsMfTuW4aA47fPntnR'
change_amount = 0.01557121 - target_amount - .00002
passphrase = b'i wish you good luck in your tribulations'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
tx_ins = []
tx_ins.append(TxIn(prev_tx, prev_index))
tx_outs = []
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
target_satoshis = int(target_amount*100000000)
tx_outs.append(TxOut(target_satoshis, script_pubkey))
h160 = decode_base58(change_address)
script_pubkey = p2pkh_script(h160)
change_satoshis = int(change_amount*100000000)
tx_outs.append(TxOut(change_satoshis, script_pubkey))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(tx_obj.sign_input(0, private_key))
print(tx_obj.serialize().hex())



68960046935564138543559644745662494926442204210008282225739377374264518817950
True
0100000001dea635e7f39dc4aa76735f939c2b79b9a79463e052a020f17d83686df050d6c2000000006b483045022100cd21de020ae0d8a43b94c94c76087646be1189fcc86b5966330284db45cbf210022007f39cbf7b6db511580c5759a4c58e08c92ddab1afc4054193b9f3c500367b3401210382e136daa948bccd9a655040820e94ab5ae32ef921f5c7dc4c0bab1a33f87ce7ffffffff0280410e00000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac30790900000000001976a91426c5dfbdc872916d51d5e3f6f7ac15ddb331b1ea88ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [301]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee

# create a transaction output for the amount and address

# sign the first input using the private key
# sign the second input using the private key 
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx

# decode_base58: We need a way to take an address and get the 20-byte hash out of it
